In [ ]:
import sys
from pathlib import Path

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
sys.path.append("..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir                = Path("..\\..\\ribasim_lumping_data\\")
network_name            = "tki_zutphen_berkel"
dhydro_basis_dir        = Path(base_dir, network_name, "2_dhydro\\") # zelfde directory, want gebruik zelfde som
dhydro_results_dir      = Path(base_dir, network_name, "2_dhydro\\") # zelfde directory, want gebruik zelfde som
results_dir             = Path(base_dir, network_name, "3_ribasim\\")

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    crs=28992
)
# network.export_to_geopackage()

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_areas")
areas_file_name         = f"{network_name}_afw_eenheden.shp"
areas_id_column         = "GFEIDENT"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
results = network.add_basis_simulation_data_set(
    set_name="winter", 
    simulation_name="tki_zuthpen_berkel_basis.dsproj_data"
)

In [ ]:
# Add simulation sets for winter, summer, etc.
network.add_simulation_data_from_set(
    set_name="winter",
    simulations_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulations_ts=pd.date_range("2000-01-06 23:00", periods=9, freq="6D"),
);

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    orifices=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
        'kdu_DR80740070', # inlaat twentekanaal
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
        'kst_ST80830045', # weir tussen uitstroompunten Zutphen
        'kst_ST80820031', 'kst_ST80820042', # test
    ], 
    edge_ids_to_include=[1801],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.generate_ribasim_network();
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'culvert': 'TabulatedRatingCurve', 
    'manual': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve',
    'edge': 'ManningResistance',
    'level_boundary_connection': 'LevelResistance'
}
# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = {
    'sto_AE80770024': 'ManningResistance', 
    'kdu_DR80740070': 'ManningResistance',
    # duikers voor wijk Zuidwijken
    'kdu_DR80940046': 'ManningResistance', 
    'kdu_DR80950043': 'ManningResistance', 
    'kdu_DR80950151': 'ManningResistance', 
}

In [ ]:
# Generate Ribasim model en exporteer naar geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion, 
    split_node_id_conversion=split_node_id_conversion
)
ribasim_model.write(results_dir / "ribasim_model")

In [ ]:
network.basins_gdf

In [ ]:
network.his_data.weirgen_state.to_dataframe().reset_index().set_index(['weirgens', 'set', 'condition']).sort_index()